## Features+Head Starter [Infer only] for HMS Brain Comp
This notebook is a fork of the excellent notebook by @DANIAL ZAKARIA, [Features+Head Starter](https://www.kaggle.com/code/nartaa/features-head-starter). I highly recommend checking out his work!

This notebook demonstrates model inference techniques, paving the way for inclusion of other models.

Looking for a beginner-friendly notebook? Look no further! My notebook, [[Train+Infer] EfficientNetB0 Starter](https://www.kaggle.com/code/minhsienweng/train-infer-efficientnetb0-starter) , guides you through both training and inference steps. Feel free to leave any comment and feedbacks.

**Change Logs**
- [Version 4] Extract abstract data generator class 
- [Version 3] (LB=0.34) Infer with an ensemble of 7 models 


# Import and Configuration

In [ ]:
import os, random
import tensorflow as tf
import tensorflow
import tensorflow.keras.backend as K
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import albumentations as albu
from scipy.signal import butter, lfilter
import librosa
from sklearn.model_selection import KFold, GroupKFold
import tensorflow.keras.backend as K, gc
from tensorflow.keras.layers import Input, Dense, Multiply, Add, Conv1D, Concatenate, LayerNormalization

In [ ]:

LOAD_MODELS_FROM = '/kaggle/input/features-head-starter-models'
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

In [ ]:
# Set random seeds
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

In [ ]:
# USE SINGLE GPU, MULTIPLE GPUS 
gpus = tf.config.list_physical_devices('GPU')
# WE USE MIXED PRECISION
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
if len(gpus)>1:
    strategy = tf.distribute.MirroredStrategy()
    print(f'Using {len(gpus)} GPUs')
else:
    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    print(f'Using {len(gpus)} GPU')

# Data generator
This data generator outputs 512x512x3, the spectrogram and eeg images are concatenated all togother in a single image. For using data augmention you can set `augment = True` when creating the train data generator.

In [ ]:
from abc import ABC, abstractmethod 
  
FEATS2 = ['Fp1','T3','C3','O1','Fp2','C4','T4','O2']
FEAT2IDX = {x:y for x,y in zip(FEATS2,range(len(FEATS2)))}
FEATS = [['Fp1','F7','T3','T5','O1'],
         ['Fp1','F3','C3','P3','O1'],
         ['Fp2','F8','T4','T6','O2'],
         ['Fp2','F4','C4','P4','O2']]
USE_PROCESSED = True # Use processed downsampled Raw EEG 
    
class BaseDataGenerator():
    'Generates data for Keras'
    def __init__(self, data, specs, eeg_specs, raw_eegs, augment, mode, data_type): 
        self.data = data
        self.augment = augment
        self.mode = mode
        self.data_type = data_type
        self.specs = specs
        self.eeg_specs = eeg_specs
        self.raw_eegs = raw_eegs
        self.on_epoch_end()
        
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        X, y = self.data_generation(index)
        if self.augment: X = self.augmentation(X)
        return X, y
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
                
    def on_epoch_end(self):
        if self.mode=='train': 
            self.data = self.data.sample(frac=1).reset_index(drop=True)
    
    # Abstract method generate data based on the trained data type
    @abstractmethod
    def data_generation(self, index):
        pass
        
    def butter_lowpass_filter(self, data, cutoff_freq=20, sampling_rate=200, order=4):
        nyquist = 0.5 * sampling_rate
        normal_cutoff = cutoff_freq / nyquist
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        filtered_data = lfilter(b, a, data, axis=0)
        return filtered_data
    
    def resize(self, img,size):
        composition = albu.Compose([
                albu.Resize(size[0],size[1])
            ])
        return composition(image=img)['image']
            
    def augmentation(self, img):
        composition = albu.Compose([
                albu.HorizontalFlip(p=0.4)
            ])
        return composition(image=img)['image']

In [ ]:
# Implementation class generates the data based on the data_type 
class DataGenerator(BaseDataGenerator):
    'Generates data for Keras'
    def __init__(self, data, specs, eeg_specs, raw_eegs, augment, 
                 mode='test', data_type='KER'): 
        super().__init__(data, specs, eeg_specs, raw_eegs, augment, mode, data_type)
    
    def data_generation(self, index):
        if self.data_type == 'KE':
            X,y = self.generate_all_specs(index)
        elif self.data_type == 'E' or self.data_type == 'K':
            X,y = self.generate_specs(index)
        elif self.data_type == 'R':
            X,y = self.generate_raw(index)
        elif self.data_type in ['ER','KR']:
            X1,y = self.generate_specs(index)
            X2,y = self.generate_raw(index)
            X = (X1,X2)
        elif self.data_type in ['KER']:
            X1,y = self.generate_all_specs(index)
            X2,y = self.generate_raw(index)
            X = (X1,X2)
        return X,y
    
    def generate_all_specs(self, index):
        X = np.zeros((512,512,3),dtype='float32')
        y = np.zeros((6,),dtype='float32')
        
        row = self.data.iloc[index]
        if self.mode=='test': 
            offset = 0
        else:
            offset = int(row.offset/2)
        
        eeg = self.eeg_specs[row.eeg_id]
        spec = self.specs[row.spec_id]
        
        imgs = [spec[offset:offset+300,k*100:(k+1)*100].T for k in [0,2,1,3]] # to match kaggle with eeg
        img = np.stack(imgs,axis=-1)
        # LOG TRANSFORM SPECTROGRAM
        img = np.clip(img,np.exp(-4),np.exp(8))
        img = np.log(img)
            
        # STANDARDIZE PER IMAGE
        img = np.nan_to_num(img, nan=0.0)    
            
        mn = img.flatten().min()
        mx = img.flatten().max()
        ep = 1e-5
        img = 255 * (img - mn) / (mx - mn + ep)
        
        X[0_0+56:100+56,:256,0] = img[:,22:-22,0] # LL_k
        X[100+56:200+56,:256,0] = img[:,22:-22,2] # RL_k
        X[0_0+56:100+56,:256,1] = img[:,22:-22,1] # LP_k
        X[100+56:200+56,:256,1] = img[:,22:-22,3] # RP_k
        X[0_0+56:100+56,:256,2] = img[:,22:-22,2] # RL_k
        X[100+56:200+56,:256,2] = img[:,22:-22,1] # LP_k
        
        X[0_0+56:100+56,256:,0] = img[:,22:-22,0] # LL_k
        X[100+56:200+56,256:,0] = img[:,22:-22,2] # RL_k
        X[0_0+56:100+56,256:,1] = img[:,22:-22,1] # LP_k
        X[100+56:200+56,256:,1] = img[:,22:-22,3] # RP_K
        
        # EEG
        img = eeg
        mn = img.flatten().min()
        mx = img.flatten().max()
        ep = 1e-5
        img = 255 * (img - mn) / (mx - mn + ep)
        X[200+56:300+56,:256,0] = img[:,22:-22,0] # LL_e
        X[300+56:400+56,:256,0] = img[:,22:-22,2] # RL_e
        X[200+56:300+56,:256,1] = img[:,22:-22,1] # LP_e
        X[300+56:400+56,:256,1] = img[:,22:-22,3] # RP_e
        X[200+56:300+56,:256,2] = img[:,22:-22,2] # RL_e
        X[300+56:400+56,:256,2] = img[:,22:-22,1] # LP_e
        
        X[200+56:300+56,256:,0] = img[:,22:-22,0] # LL_e
        X[300+56:400+56,256:,0] = img[:,22:-22,2] # RL_e
        X[200+56:300+56,256:,1] = img[:,22:-22,1] # LP_e
        X[300+56:400+56,256:,1] = img[:,22:-22,3] # RP_e

        if self.mode!='test':
            y[:] = row[TARGETS]
        
        return X,y
    
    def generate_specs(self, index):
        X = np.zeros((512,512,3),dtype='float32')
        y = np.zeros((6,),dtype='float32')
        
        row = self.data.iloc[index]
        if self.mode=='test': 
            offset = 0
        else:
            offset = int(row.offset/2)
            
        if self.data_type in ['E','ER']:
            img = self.eeg_specs[row.eeg_id]
        elif self.data_type in ['K','KR']:
            spec = self.specs[row.spec_id]
            imgs = [spec[offset:offset+300,k*100:(k+1)*100].T for k in [0,2,1,3]] # to match kaggle with eeg
            img = np.stack(imgs,axis=-1)
            # LOG TRANSFORM SPECTROGRAM
            img = np.clip(img,np.exp(-4),np.exp(8))
            img = np.log(img)
            
            # STANDARDIZE PER IMAGE
            img = np.nan_to_num(img, nan=0.0)    
            
        mn = img.flatten().min()
        mx = img.flatten().max()
        ep = 1e-5
        img = 255 * (img - mn) / (mx - mn + ep)
        
        X[0_0+56:100+56,:256,0] = img[:,22:-22,0]
        X[100+56:200+56,:256,0] = img[:,22:-22,2]
        X[0_0+56:100+56,:256,1] = img[:,22:-22,1]
        X[100+56:200+56,:256,1] = img[:,22:-22,3]
        X[0_0+56:100+56,:256,2] = img[:,22:-22,2]
        X[100+56:200+56,:256,2] = img[:,22:-22,1]
        
        X[0_0+56:100+56,256:,0] = img[:,22:-22,0]
        X[100+56:200+56,256:,0] = img[:,22:-22,1]
        X[0_0+56:100+56,256:,1] = img[:,22:-22,2]
        X[100+56:200+56,256:,1] = img[:,22:-22,3]
        
        X[200+56:300+56,:256,0] = img[:,22:-22,0]
        X[300+56:400+56,:256,0] = img[:,22:-22,1]
        X[200+56:300+56,:256,1] = img[:,22:-22,2]
        X[300+56:400+56,:256,1] = img[:,22:-22,3]
        X[200+56:300+56,:256,2] = img[:,22:-22,3]
        X[300+56:400+56,:256,2] = img[:,22:-22,2]
        
        X[200+56:300+56,256:,0] = img[:,22:-22,0]
        X[300+56:400+56,256:,0] = img[:,22:-22,2]
        X[200+56:300+56,256:,1] = img[:,22:-22,1]
        X[300+56:400+56,256:,1] = img[:,22:-22,3]
        
        if self.mode!='test':
            y[:] = row[TARGETS]
        
        return X,y
    
    def generate_raw(self,index):
        if USE_PROCESSED and self.mode!='test':
            X = np.zeros((2_000,8),dtype='float32')
            y = np.zeros((6,),dtype='float32')
            row = self.data.iloc[index]
            X = self.raw_eegs[row.eeg_id]
            y[:] = row[TARGETS]
            return X,y
        
        X = np.zeros((10_000,8),dtype='float32')
        y = np.zeros((6,),dtype='float32')
        
        row = self.data.iloc[index]
        eeg = self.raw_eegs[row.eeg_id]
            
        # FEATURE ENGINEER
        X[:,0] = eeg[:,FEAT2IDX['Fp1']] - eeg[:,FEAT2IDX['T3']]
        X[:,1] = eeg[:,FEAT2IDX['T3']] - eeg[:,FEAT2IDX['O1']]
            
        X[:,2] = eeg[:,FEAT2IDX['Fp1']] - eeg[:,FEAT2IDX['C3']]
        X[:,3] = eeg[:,FEAT2IDX['C3']] - eeg[:,FEAT2IDX['O1']]
            
        X[:,4] = eeg[:,FEAT2IDX['Fp2']] - eeg[:,FEAT2IDX['C4']]
        X[:,5] = eeg[:,FEAT2IDX['C4']] - eeg[:,FEAT2IDX['O2']]
            
        X[:,6] = eeg[:,FEAT2IDX['Fp2']] - eeg[:,FEAT2IDX['T4']]
        X[:,7] = eeg[:,FEAT2IDX['T4']] - eeg[:,FEAT2IDX['O2']]
            
        # STANDARDIZE
        X = np.clip(X,-1024,1024)
        X = np.nan_to_num(X, nan=0) / 32.0
            
        # BUTTER LOW-PASS FILTER
        X = self.butter_lowpass_filter(X)
        # Downsample
        X = X[::5,:]
        
        if self.mode!='test':
            y[:] = row[TARGETS]
                
        return X,y

# Load test data
Infer the test data and create a `submission.csv` file.

In [ ]:
def spectrogram_from_eeg(parquet_path):    
    # LOAD MIDDLE 50 SECONDS OF EEG SERIES
    eeg = pd.read_parquet(parquet_path)
    middle = (len(eeg)-10_000)//2
    eeg = eeg.iloc[middle:middle+10_000]
    
    # VARIABLE TO HOLD SPECTROGRAM
    img = np.zeros((100,300,4) ,dtype='float32')

    for k in range(4):
        COLS = FEATS[k]
        
        for kk in range(4):
            # FILL NANS
            x1 = eeg[COLS[kk]].values
            x2 = eeg[COLS[kk+1]].values
            m = np.nanmean(x1)
            if np.isnan(x1).mean()<1: 
                x1 = np.nan_to_num(x1,nan=m)
            else: 
                x1[:] = 0
            m = np.nanmean(x2)
            if np.isnan(x2).mean()<1: 
                x2 = np.nan_to_num(x2,nan=m)
            else: 
                x2[:] = 0
                
            # COMPUTE PAIR DIFFERENCES
            x = x1 - x2

            # RAW SPECTROGRAM
            mel_spec = librosa.feature.melspectrogram(y=x, sr=200, hop_length=len(x)//300, 
                                  n_fft=1024, n_mels=100, fmin=0, fmax=20, win_length=128)
            
            # LOG TRANSFORM
            width = (mel_spec.shape[1]//30)*30
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[:,:width]
            img[:,:,k] += mel_spec_db
                
        # AVERAGE THE 4 MONTAGE DIFFERENCES
        img[:,:,k] /= 4.0
    return img
# Read EEG from par files
def eeg_from_parquet(parquet_path):
    eeg = pd.read_parquet(parquet_path, columns=FEATS2)
    rows = len(eeg)
    offset = (rows-10_000)//2
    eeg = eeg.iloc[offset:offset+10_000]
    data = np.zeros((10_000,len(FEATS2)))
    for j,col in enumerate(FEATS2):
        
        # FILL NAN
        x = eeg[col].values.astype('float32')
        m = np.nanmean(x)
        if np.isnan(x).mean()<1: x = np.nan_to_num(x,nan=m)
        else: x[:] = 0
        
        data[:,j] = x

    return data

In [ ]:
# Load testing features
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
# Rename
test = test.rename({'spectrogram_id':'spec_id'},axis=1)
print('Test shape',test.shape)
test.head()

In [ ]:
# Read all spectrograms
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms'
files = os.listdir(PATH)
print(f'There are {len(files)} test spectrogram parquets')
specs = {}
for i,f in enumerate(files):
    tmp = pd.read_parquet(f'{PATH}/{f}')
    name = int(f.split('.')[0])
    specs[name] = tmp.iloc[:,1:].values

In [ ]:
# Read all EEG Spectrograms
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs'
DISPLAY = 0
EEG_IDS = test.eeg_id.unique()
eeg_specs = {}
print('Converting Test EEG to Spectrograms...')
for i,eeg_id in enumerate(EEG_IDS):
    # CREATE SPECTROGRAM FROM EEG PARQUET
    eeg_specs[eeg_id] = spectrogram_from_eeg(f'{PATH}/{eeg_id}.parquet')

In [ ]:
# Read all RAW EEG Signals
raw_eegs = {}
EEG_IDS = test.eeg_id.unique()
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs'
print('Processing Test EEG parquets...')
for i,eeg_id in enumerate(EEG_IDS):
    # SAVE EEG TO PYTHON DICTIONARY OF NUMPY ARRAYS
    raw_eegs[eeg_id] = eeg_from_parquet(f'{PATH}/{eeg_id}.parquet')

In [ ]:
def create_dataset(data, mode='test', data_type='KER', batch_size=8,  
                    augment=False, specs=specs, eeg_specs=eeg_specs, raw_eegs=raw_eegs):    
    if data_type in ['K','E','KE']: 
        inp = tf.TensorSpec(shape=(512,512,3), dtype=tf.float32)
    elif data_type in ['KR','ER','KER']:
        inp = (tf.TensorSpec(shape=(512,512,3), dtype=tf.float32), 
               tf.TensorSpec(shape=(2000,8), dtype=tf.float32))
    elif data_type in ['R']:
        inp = tf.TensorSpec(shape=(2000,8), dtype=tf.float32)

    output_signature = (inp, tf.TensorSpec(shape=(6,), dtype=tf.float32))
    
    # Create the data generator
    gen = DataGenerator(data, specs, eeg_specs, raw_eegs, augment, mode, data_type)
    # Create the dataset from data generator
    dataset = tf.data.Dataset.from_generator(generator=gen,
                                             output_signature=output_signature).batch(batch_size * strategy.num_replicas_in_sync)
    return dataset

In [ ]:
preds = []

test_dataset_K = create_dataset(test, data_type='K', mode='test')
test_dataset_E = create_dataset(test, data_type='E', mode='test')
test_dataset_R = create_dataset(test, data_type='R', mode='test')
test_dataset_KE = create_dataset(test, data_type='KE', mode='test')
test_dataset_KR = create_dataset(test, data_type='KR', mode='test')
test_dataset_ER = create_dataset(test, data_type='ER', mode='test')
test_dataset_KER = create_dataset(test, data_type='KER', mode='test')

# Infer with an individual model

In [ ]:
DATA_TYPE = 'KER' # K|E|R|KE|KR|ER|KER
# Submission ON TEST without ensemble
def preds_without_ensemble(VER=50):
    preds = []
    
    test_dataset = dataset(test,mode='test',specs=spectrograms2, eeg_specs=all_eegs2, raw_eegs=all_raw_eegs2)
    model = build_model()

    for i in range(5):
        print(f'Fold {i+1}')
        model.load_weights(f'{LOAD_MODELS_FROM}/model_{DATA_TYPE}_{VER}_{i}.weights.h5')
        pred = model.predict(test_dataset, verbose=1)
        preds.append(pred)
        
    pred = np.mean(preds,axis=0)
    print('Test preds shape',pred.shape)

# Infer with ensemble of models

In [ ]:
# Setup for ensemble
LOAD_BACKBONE_FROM = '/kaggle/input/efficientnetb-tf-keras/EfficientNetB2.h5'
ENSEMBLE = True
LBs = [0.41,0.39,0.41,0.37,0.39,0.38,0.36] # K|E|R|KE|KR|ER|KER for weighted ensemble we use LBs of each model
VER_K = 43 # Kaggle's spectrogram model version
VER_E = 42 # EEG's spectrogram model version
VER_R = 37 # EEG's Raw wavenet model version, trained on single GPU
VER_KE = 47 # Kaggle's and EEG's spectrogram model version
VER_KR = 48 # Kaggle's spectrogram and Raw model version
VER_ER = 49 # EEG's spectrogram and Raw model version
VER_KER = 50 # EEG's, Kaggle's spectrograms and Raw model version

In [ ]:
class SPECModel:
    def __init__(self, hybrid=False):
        self.hybrid = hybrid
        self.build_model()
        
    def get_model(self):
        return self.model
        
    def build_model(self):
        inp = tf.keras.layers.Input((512,512,3))
        base_model = load_model(f'{LOAD_BACKBONE_FROM}')    
        x = base_model(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        if not self.hybrid:
            x = tf.keras.layers.Dense(6,activation='softmax', dtype='float32')(x)
        # Create the model
        self.model = tf.keras.Model(inputs=inp, outputs=x)
        opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
        loss = tf.keras.losses.KLDivergence()
        self.model.compile(loss=loss, optimizer=opt)

In [ ]:
class WAVEModel(SPECModel):
    def __init__(self, hybrid=False):
        self.hybrid = hybrid
        self.build_model()
        
    def wave_block(self, x, filters, kernel_size, n):
        dilation_rates = [2**i for i in range(n)]
        x = Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same', 
                              activation = 'tanh', 
                              dilation_rate = dilation_rate)(x)
            sigm_out = Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'sigmoid', 
                              dilation_rate = dilation_rate)(x)
            x = Multiply()([tanh_out, sigm_out])
            x = Conv1D(filters = filters,
                       kernel_size = 1,
                       padding = 'same')(x)
            res_x = Add()([res_x, x])
        return res_x
        
    def build_model(self):
        # INPUT 
        inp = tf.keras.Input(shape=(2_000,8))

        ############
        # FEATURE EXTRACTION SUB MODEL
        inp2 = tf.keras.Input(shape=(2_000,1))
        x = self.wave_block(inp2, 8, 4, 6)
        x = self.wave_block(x, 16, 4, 6)
        x = self.wave_block(x, 32, 4, 6)
        x = self.wave_block(x, 64, 4, 6)
        model2 = tf.keras.Model(inputs=inp2, outputs=x)
        ###########

        # LEFT TEMPORAL CHAIN
        x1 = model2(inp[:,:,0:1])
        x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
        x2 = model2(inp[:,:,1:2])
        x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
        z1 = tf.keras.layers.Average()([x1,x2])

        # LEFT PARASAGITTAL CHAIN
        x1 = model2(inp[:,:,2:3])
        x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
        x2 = model2(inp[:,:,3:4])
        x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
        z2 = tf.keras.layers.Average()([x1,x2])

        # RIGHT PARASAGITTAL CHAIN
        x1 = model2(inp[:,:,4:5])
        x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
        x2 = model2(inp[:,:,5:6])
        x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
        z3 = tf.keras.layers.Average()([x1,x2])

        # RIGHT TEMPORAL CHAIN
        x1 = model2(inp[:,:,6:7])
        x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
        x2 = model2(inp[:,:,7:8])
        x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
        z4 = tf.keras.layers.Average()([x1,x2])

        # COMBINE CHAINS
        y = tf.keras.layers.Concatenate()([z1,z2,z3,z4])
        if not self.hybrid:
            y = tf.keras.layers.Dense(64, activation='relu')(y)
            y = tf.keras.layers.Dense(6,activation='softmax', dtype='float32')(y)

        # COMPILE MODEL
        self.model = tf.keras.Model(inputs=inp, outputs=y)
        opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
        loss = tf.keras.losses.KLDivergence()
        self.model.compile(loss=loss, optimizer = opt)
    

In [ ]:
class HYBRIDModel(SPECModel):
    def __init__(self):
        self.build_model()      
        
    def build_model(self):
        model_spec = SPECModel(True).model
        model_wave = WAVEModel(True).model
        inputs = [model_spec.input, model_wave.input]
        x = [model_spec.output, model_wave.output]
        x = tf.keras.layers.Concatenate()(x)
        x = tf.keras.layers.Dense(6,activation='softmax', dtype='float32')(x)

        # COMPILE MODEL
        self.model = tf.keras.Model(inputs=inputs, outputs=x)
        opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
        loss = tf.keras.losses.KLDivergence()
        self.model.compile(loss=loss, optimizer = opt)

In [ ]:
# Submission ON TEST with ensemble
def preds_with_ensemble():
    preds = []
    # LB SCORE WEIGHTS FOR EACH MODEL
    lbs = 1 - np.array(LBs)
    weights = lbs/lbs.sum()
    model_spec = SPECModel().get_model()
    model_wave = WAVEModel().get_model()
    model_hybrid = HYBRIDModel().get_model()
    print(model_spec)
    
    
    for i in range(5):
        print(f'Fold {i+1}')
        # Kaggle's spectrogram model 
        model_spec.load_weights(f'{LOAD_MODELS_FROM}/model_K_{VER_K}_{i}.weights.h5')
        pred_K = model_spec.predict(test_dataset_K, verbose=1)
        # EEG's spectrogram model
        model_spec.load_weights(f'{LOAD_MODELS_FROM}/model_E_{VER_E}_{i}.weights.h5')
        pred_E = model_spec.predict(test_dataset_E, verbose=1)
        # EEG's Raw wavenet model
        model_wave.load_weights(f'{LOAD_MODELS_FROM}/model_R_{VER_R}_{i}.weights.h5')
        pred_R = model_wave.predict(test_dataset_R, verbose=1)
        # Kaggle's and EEG's spectrogram model
        model_spec.load_weights(f'{LOAD_MODELS_FROM}/model_KE_{VER_KE}_{i}.weights.h5')
        pred_KE = model_spec.predict(test_dataset_KE, verbose=1)
        # Kaggle's spectrogram and Raw model
        model_hybrid.load_weights(f'{LOAD_MODELS_FROM}/model_KR_{VER_KR}_{i}.weights.h5')
        pred_KR = model_hybrid.predict(test_dataset_KR, verbose=1)
        # EEG's spectrogram and Raw model
        model_hybrid.load_weights(f'{LOAD_MODELS_FROM}/model_ER_{VER_ER}_{i}.weights.h5')
        pred_ER = model_hybrid.predict(test_dataset_ER, verbose=1)
        # EEG's, Kaggle's spectrograms and Raw model
        model_hybrid.load_weights(f'{LOAD_MODELS_FROM}/model_KER_{VER_KER}_{i}.weights.h5')
        pred_KER = model_hybrid.predict(test_dataset_KER, verbose=1)
        # Combine the predictions from all the model with different weights 
        pred = np.array([pred_K,pred_E,pred_R,pred_KE,pred_KR,pred_ER,pred_KER])
        pred = np.average(pred,axis=0,weights=weights)
        preds.append(pred)
        
    pred = np.mean(preds,axis=0)
    return pred
# Prediction with 
pred = preds_with_ensemble()
print('Test preds shape',pred.shape)

# Create Submission CSV

In [ ]:
sub = pd.DataFrame({'eeg_id':test.eeg_id.values})
sub[TARGETS] = pred
sub.to_csv('submission.csv',index=False)
print('Submissionn shape',sub.shape)
print()
print(sub.head().to_string())

In [ ]:
# SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
print(sub.iloc[:,-6:].sum(axis=1).to_string())